#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [250]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=123
)

In [251]:
def entropy_func(class_count, num_samples):
    probability = class_count / num_samples
    return -(probability) * math.log(probability, 2)


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size

    # assuming that group_classes is an array of classes
    def group_entropy(self):
        # determine classes of data ponits and calculate the entropy of the whole group
        _, counts = np.unique(self.group_classes, return_counts=True)
        entropy = np.sum([entropy_func(count, len(self)) for count in counts])
        return entropy

In [252]:
class Node:
    """
    split feature is the index of the feature that gives the best information gain
    split_val is the value that gives the best information gain
    depth is distance between the node and the root node
    child_node_a is a node that collects the data points that meet split requirement
    child_node_b is a node that collects the data points that do not meet split requirement
    val is the predicted class for the data points that reach the node, if val is None, then the Node is not a leaf
    if val is not None, then the Node is a leaf
    """
    def __init__(
        self,
        split_feature,
        split_val,
        depth=None,
        child_node_a=None,
        child_node_b=None,
        val=None,
    ):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    """
    Calculate the predicted value recuresively, using children nodes
    (if the node is a leaf return value, else pass the data point to a proper child node)
    """
    def predict(self, data):
        if not self.val == None:
            return self.val
        elif data[self.split_feature] < self.split_val:
            return self.child_node_a.predict(data)
        else:
            return self.child_node_b.predict(data)

In [253]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    """
    Calculate the split entropy of two groups (a group is an array of possible classes)
    """
    @staticmethod
    def get_split_entropy(group_a, group_b):
        return (group_a.entropy * len(group_a) + group_b.entropy * len(group_b)) / (
            len(group_a) + len(group_b)
        )
        
    """
    Calculate the information gain of a given split
    """
    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        return parent_group.entropy - self.get_split_entropy(
            child_group_a, child_group_b
        )

    """
    Find the best possible split for a given feature
    """
    def get_best_feature_split(self, feature_values, classes):
        best_information_gain = 0
        best_feature_split = None
        for value in feature_values:
            indices_meeting_condition = [
                i for i in range(len(classes)) if feature_values[i] < value
            ]
            indices_not_meeting_condition = [
                i for i in range(len(classes)) if not i in indices_meeting_condition
            ]
            informtion_gain = self.get_information_gain(
                Group(classes),
                Group(classes[indices_meeting_condition]),
                Group(classes[indices_not_meeting_condition]),
            )
            if informtion_gain > best_information_gain:
                best_information_gain = informtion_gain
                best_feature_split = value
        return best_information_gain, best_feature_split


    """
    Find the best possible split for the data set 
    (calculates all the possible splits for all of the possible features 
    returns the pair that gives the largest information gain)
    """
    def get_best_split(self, data, classes):
        best_information_gain = 0
        best_feature = None
        best_split = None
        for i in range(len(data[0])):
            information_gain, split_value = self.get_best_feature_split(
                data[:, i], classes
            )
            if information_gain > best_information_gain:
                best_information_gain = information_gain
                best_feature = i
                best_split = split_value
        return best_feature, best_split

    """
    Build a tree from given data recursevily, returns the root node.
    
    """
    def build_tree(self, data, classes, depth=0):
        if len(np.unique(classes)) == 1: #Check if only one possible class in data
            return Node(None, None, val=classes[0])
        if all(all(element == data[0]) for element in data):
            return Node(None, None, val=Counter(classes).most_common()[0][0]) #Check if furthur division makes sense (given data points are not all the same)
        if depth == self.max_depth:
            return Node(None, None, val=Counter(classes).most_common()[0][0]) #Check if depth limit reached
        best_feature, best_split = self.get_best_split(data, classes) #Get the best possbile split
        indices_meeting_condition = [
            i for i in range(len(data)) if data[i][best_feature] < best_split
        ]
        indices_not_meeting_condition = [
            i for i in range(len(data)) if not i in indices_meeting_condition
        ]
        child_a_value = None
        child_b_value = None
        if len(indices_meeting_condition) == 0: #Check if child_node_a will recieve any data points
            child_a_value = Counter(classes).most_common()[0][0] #if not determine the value of child_node_a
        if len(indices_not_meeting_condition) == 0: #Check if child_node_b will recieve any data points
            child_b_value = Counter(classes).most_common()[0][0] #if not determine the value of child_node_B

        #if child_node_a will not recieve data points, mark the child_node_a as a leaf, 
        #else calculate children nodes of child_node_a 
        child_node_a = (
            Node(None, None, val=child_a_value)
            if child_a_value
            else self.build_tree(
                data[indices_meeting_condition],
                classes[indices_meeting_condition],
                depth + 1,
            )
        )
        #if child_node_b will not recieve data points, mark the child_node_b as a leaf, 
        #else calculate children nodes of child_node_b
        child_node_b = (
            Node(None, None, val=child_b_value)
            if child_b_value
            else self.build_tree(
                data[indices_not_meeting_condition],
                classes[indices_not_meeting_condition],
                depth + 1,
            )
        )
        return Node(best_feature, best_split, depth, child_node_a, child_node_b) #Return the root node

    def predict(self, data):
        return self.tree.predict(data)

In [254]:
iris = load_iris()
x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=123
)
dc = DecisionTreeClassifier(3)
dc.tree = dc.build_tree(x_train, y_train)
good_predictions = 0
print("TESTING ID3:")
print("Test size: 0.1")
print("Random state = 123\n")
print("True values vs Predicred values:")
counter = 1
for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    print(f"{counter}. true value: {gt}; predicted value: {prediction}")
    if prediction == gt:
        good_predictions += 1
    counter += 1
print(f"Accuracy: {good_predictions/len(y_test)}")

TESTING ID3:
Test size: 0.1
Random state = 123

True values vs Predicred values:
1. true value: 1; predicted value: 2
2. true value: 2; predicted value: 2
3. true value: 2; predicted value: 2
4. true value: 1; predicted value: 1
5. true value: 0; predicted value: 0
6. true value: 2; predicted value: 2
7. true value: 1; predicted value: 1
8. true value: 0; predicted value: 0
9. true value: 0; predicted value: 0
10. true value: 1; predicted value: 1
11. true value: 2; predicted value: 2
12. true value: 0; predicted value: 0
13. true value: 1; predicted value: 1
14. true value: 2; predicted value: 2
15. true value: 2; predicted value: 2
Accuracy: 0.9333333333333333


In [255]:
def run_experiment(tree_depth, test_size, random_state=None, verbose=True):
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, random_state=random_state
    )
    dc = DecisionTreeClassifier(tree_depth)
    dc.tree = dc.build_tree(x_train, y_train)
    good_predictions = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        if prediction == gt:
            good_predictions += 1
    accuracy = good_predictions / len(y_test)
    if verbose:
        print("TESTING ID3:")
        print(f"Tree depth {tree_depth}")
        print(f"Test size: {test_size}")
        print(f"Random state = {random_state}")
        print(f"Accuracy = {accuracy}\n")
    else:
        return accuracy

In [256]:
run_experiment(3, 0.1, 123)
run_experiment(2, 0.1, 123)
run_experiment(1, 0.1, 123)
run_experiment(5, 0.1, 123)
run_experiment(10, 0.1, 123)


TESTING ID3:
Tree depth 3
Test size: 0.1
Random state = 123
Accuracy = 0.9333333333333333

TESTING ID3:
Tree depth 2
Test size: 0.1
Random state = 123
Accuracy = 0.8666666666666667

TESTING ID3:
Tree depth 1
Test size: 0.1
Random state = 123
Accuracy = 0.6

TESTING ID3:
Tree depth 5
Test size: 0.1
Random state = 123
Accuracy = 0.9333333333333333

TESTING ID3:
Tree depth 10
Test size: 0.1
Random state = 123
Accuracy = 0.9333333333333333



In [257]:
run_experiment(3, 0.05, 123)
run_experiment(3, 0.2, 123)
run_experiment(3, 0.3, 123)
run_experiment(3, 0.5, 123)
run_experiment(3, 0.8, 123)
run_experiment(3, 0.9, 123)
run_experiment(3, 0.95, 123)

TESTING ID3:
Tree depth 3
Test size: 0.05
Random state = 123
Accuracy = 0.875

TESTING ID3:
Tree depth 3
Test size: 0.2
Random state = 123
Accuracy = 0.9666666666666667

TESTING ID3:
Tree depth 3
Test size: 0.3
Random state = 123
Accuracy = 0.9333333333333333

TESTING ID3:
Tree depth 3
Test size: 0.5
Random state = 123
Accuracy = 0.9733333333333334

TESTING ID3:
Tree depth 3
Test size: 0.8
Random state = 123
Accuracy = 0.8166666666666667

TESTING ID3:
Tree depth 3
Test size: 0.9
Random state = 123
Accuracy = 0.837037037037037

TESTING ID3:
Tree depth 3
Test size: 0.95
Random state = 123
Accuracy = 0.7622377622377622



In [258]:
run_experiment(2, 0.05, 123)
run_experiment(2, 0.2, 123)
run_experiment(2, 0.3, 123)
run_experiment(2, 0.5, 123)
run_experiment(2, 0.8, 123)
run_experiment(2, 0.9, 123)
run_experiment(2, 0.95, 123)

TESTING ID3:
Tree depth 2
Test size: 0.05
Random state = 123
Accuracy = 0.75

TESTING ID3:
Tree depth 2
Test size: 0.2
Random state = 123
Accuracy = 0.9666666666666667

TESTING ID3:
Tree depth 2
Test size: 0.3
Random state = 123
Accuracy = 0.9555555555555556

TESTING ID3:
Tree depth 2
Test size: 0.5
Random state = 123
Accuracy = 0.96

TESTING ID3:
Tree depth 2
Test size: 0.8
Random state = 123
Accuracy = 0.8166666666666667

TESTING ID3:
Tree depth 2
Test size: 0.9
Random state = 123
Accuracy = 0.837037037037037

TESTING ID3:
Tree depth 2
Test size: 0.95
Random state = 123
Accuracy = 0.7622377622377622



In [259]:
repetitions = 5
for _ in range(repetitions):
    run_experiment(3, 0.1)

TESTING ID3:
Tree depth 3
Test size: 0.1
Random state = None
Accuracy = 0.8666666666666667

TESTING ID3:
Tree depth 3
Test size: 0.1
Random state = None
Accuracy = 0.9333333333333333

TESTING ID3:
Tree depth 3
Test size: 0.1
Random state = None
Accuracy = 0.9333333333333333

TESTING ID3:
Tree depth 3
Test size: 0.1
Random state = None
Accuracy = 1.0

TESTING ID3:
Tree depth 3
Test size: 0.1
Random state = None
Accuracy = 1.0



In [260]:
repetitions = 100
depth_three_accuracy = 0.0
for _ in range(repetitions):
    depth_three_accuracy += run_experiment(3, 0.1, verbose=False)

depth_two_accuracy = 0.0
for _ in range(repetitions):
    depth_two_accuracy += run_experiment(2, 0.1, verbose=False)
    
depth_one_accuracy = 0.0
for _ in range(repetitions):
    depth_one_accuracy += run_experiment(1, 0.1, verbose=False)
    
print(f"Average depth three accuracy: {depth_three_accuracy/repetitions}")
print(f"Average depth two accuracy: {depth_two_accuracy/repetitions}")
print(f"Average depth one accuracy: {depth_one_accuracy/repetitions}")


Average depth three accuracy: 0.9479999999999997
Average depth two accuracy: 0.9366666666666665
Average depth one accuracy: 0.5986666666666669


# Wnioski
Drzewo decyzyjne ID3 osiąga dobrą skuteczność, jeśli dobrana jest głębokość odpowiednia do złożoności danych- tzn. im więcej wymiarów posiadają dane, tym większej głębokości drzewo będzie potrzebować, aby móć poprawnie przewidywać klasy. Zazwyczaj drzewo z większą głębokością osiągnie lepszą skuteczność przewidywań od drzewa z mniejszą głębokością, jednak jeśli maksymalna głębokość jest dostatecznie duża, jej zwiększanie nie przynosi żadnych rezultatów, gdyż drzewo przestaje być rozbudowywane ze względu na inne kryteria stopu. Rozmiar zbioru tesującego powinien być reprezentatywnym pozdbiorem dziedziny (wybrany zgodnie z rozkładem losowym i odpowiednio duży). W przeciwnym wypadku model nie będzie wytrenowany odpowiednio i nie będzie mógł skutecznie przewidywać klas przykładów.